# A4 War

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt
from fractions import Fraction

## Simulate

In [2]:
CRT = pd.DataFrame({Fraction(2, 7): ["N/E", "N/E", "N/E", "N/E", "N/E", "N/E", "N/E", "N/E", "N/E", "N/E", "N/E", "D"],
                    Fraction(1, 3): ["N/E", "N/E", "N/E", "N/E", "N/E", "N/E", "N/E", "N/E", "N/E", "N/E", "D", "D"],
                    Fraction(2, 5): ["N/E", "N/E", "N/E", "N/E", "N/E", "N/E", "N/E", "N/E", "N/E", "D", "D", "X"],
                    Fraction(1, 2): ["N/E", "N/E", "N/E", "N/E", "N/E", "N/E", "N/E", "N/E", "D", "D", "D", "X"],
                    Fraction(3, 4): ["N/E", "N/E", "N/E", "N/E", "N/E", "N/E", "D", "D", "D", "X", "X", "X"],
                    Fraction(1, 1): ["N/E", "N/E", "N/E", "N/E", "D", "D", "D", "D", "X", "X", "X", "X"],
                    Fraction(3, 2): ["N/E", "N/E", "N/E", "D", "D", "D", "D", "X", "X", "X", "X", "X"],
                    Fraction(2, 1): ["N/E", "N/E", "D", "D", "D", "D", "X", "X", "X", "X", "X", "X"],
                    Fraction(5, 2): ["N/E", "D", "D", "D", "D", "X", "X", "X", "X", "X", "X", "X"],
                    Fraction(3, 1): ["D", "D", "D", "D", "X", "X", "X", "X", "X", "X", "X", "X"],
                    Fraction(7, 2): ["D", "D", "D", "X", "X", "X", "X", "X", "X", "X", "X", "X"],
                    Fraction(4, 1): ["D", "D", "X", "X", "X", "X", "X", "X", "X", "X", "X", "X"],
                    Fraction(9, 2): ["D", "X", "X", "X", "X", "X", "X", "X", "X", "X", "X", "X"]},
                   index=range(1, 13))
CRT

,2/7,1/3,2/5,1/2,3/4,1,3/2,2,5/2,3,7/2,4,9/2
1,N/E,N/E,N/E,N/E,N/E,N/E,N/E,N/E,N/E,D,D,D,D
2,N/E,N/E,N/E,N/E,N/E,N/E,N/E,N/E,D,D,D,D,X
3,N/E,N/E,N/E,N/E,N/E,N/E,N/E,D,D,D,D,X,X
4,N/E,N/E,N/E,N/E,N/E,N/E,D,D,D,D,X,X,X
5,N/E,N/E,N/E,N/E,N/E,D,D,D,D,X,X,X,X
6,N/E,N/E,N/E,N/E,N/E,D,D,D,X,X,X,X,X
7,N/E,N/E,N/E,N/E,D,D,D,X,X,X,X,X,X
8,N/E,N/E,N/E,N/E,D,D,X,X,X,X,X,X,X
9,N/E,N/E,N/E,D,D,X,X,X,X,X,X,X,X
10,N/E,N/E,D,D,X,X,X,X,X,X,X,X,X


In [3]:
def crt_column(attack, defence):
    """
    returns the closest ratio on the CRT lookup table for the passed attack and defence value
    """
    ad_fraction = Fraction(attack, defence)
    return min(CRT.columns.values, key=lambda x: abs(x - ad_fraction))

In [4]:
crt_column(10, 3)

Fraction(7, 2)

In [5]:
acc = []
for column in CRT:
    acc.append(CRT[column].value_counts())

P_CRT = (pd.DataFrame(acc)
         .fillna(0))/12         
P_CRT

,N/E,D,X
2/7,0.916667,0.083333,0.000000
1/3,0.833333,0.166667,0.000000
2/5,0.750000,0.166667,0.083333
1/2,0.666667,0.250000,0.083333
3/4,0.500000,0.250000,0.250000
1,0.333333,0.333333,0.333333
3/2,0.250000,0.333333,0.416667
2,0.166667,0.333333,0.500000
5/2,0.083333,0.333333,0.583333
3,0.000000,0.333333,0.666667


In [6]:
acc = []
for ratio in CRT.columns:
    P_NE = stats.multinomial(3, P_CRT.loc[ratio]).pmf([3, 0, 0]) +\
           stats.multinomial(3, P_CRT.loc[ratio]).pmf([2, 1, 0]) +\
           stats.multinomial(3, P_CRT.loc[ratio]).pmf([2, 0, 1]) +\
           stats.multinomial(4, P_CRT.loc[ratio]).pmf([2, 1, 1])
    
    P_D = stats.multinomial(3, P_CRT.loc[ratio]).pmf([0, 3, 0]) +\
          stats.multinomial(3, P_CRT.loc[ratio]).pmf([1, 2, 0]) +\
          stats.multinomial(3, P_CRT.loc[ratio]).pmf([0, 2, 0]) +\
          stats.multinomial(4, P_CRT.loc[ratio]).pmf([1, 2, 1])
    
    P_X = stats.multinomial(3, P_CRT.loc[ratio]).pmf([0, 0, 3]) +\
          stats.multinomial(3, P_CRT.loc[ratio]).pmf([1, 0, 2]) +\
          stats.multinomial(3, P_CRT.loc[ratio]).pmf([0, 1, 2]) +\
          stats.multinomial(4, P_CRT.loc[ratio]).pmf([1, 1, 2])
    
    acc.append({"N/E": P_NE, "D": P_D, "X": P_X})

P_CRT_3 = pd.DataFrame(acc, index=P_CRT.index)
P_CRT_3    

,N/E,D,X
2/7,0.980324,0.019676,0.000000
1/3,0.925926,0.074074,0.000000
2/5,0.937500,0.087963,0.030093
1/2,0.851852,0.182292,0.033565
3/4,0.687500,0.203125,0.250000
1,0.407407,0.296296,0.407407
3/2,0.260417,0.259259,0.549769
2,0.129630,0.203704,0.666667
5/2,0.035880,0.129630,0.737269
3,0.000000,0.037037,0.740741
